<a href="https://colab.research.google.com/github/Aditi1K/SOI_storyForge/blob/main/Untitled7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate moviepy gtts

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
from diffusers import StableDiffusionPipeline
import torch
import os
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
from gtts import gTTS
from transformers import pipeline

In [ ]:
def split_story_into_sentences(story):
    """
    Split the story into sentences using a custom approach.

    :param story: The full story as a single string.
    :return: A list of sentences.
    """
    sentences = story.split('. ')
    sentences = [sentence.strip() + '.' if not sentence.endswith('.') else sentence.strip() for sentence in sentences]
    return sentences

def add_text_to_image(image, text, font_size=24):
    """
    Add text to an image.

    :param image: The image to add text to.
    :param text: The text to add.
    :param font_size: The font size of the text.
    :return: An Image object with the text added.
    """
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    lines = textwrap.fill(text, width=40)
    text_width, text_height = draw.textsize(lines, font=font)
    x = (image.width - text_width) // 2
    y = image.height - text_height - 10

    draw.text((x, y), lines, font=font, fill='black')

    return image

def generate_images_from_story(story, output_dir):
    """
    Generate images from story sentences and save them to the output directory.

    :param story: The full story as a single string.
    :param output_dir: The directory to save the images.
    """
    sentences = split_story_into_sentences(story)

    pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base", torch_dtype=torch.float16)
    pipe.to("cuda")

    for i, sentence in enumerate(sentences):
        image = pipe(sentence).images[0]
        image = add_text_to_image(image, sentence)

        image_path = f"{output_dir}/sentence_{i+1:02d}.png"
        image.save(image_path)
        print(f"Saved image: {image_path}")

        tts = gTTS(sentence)
        audio_path = f"{output_dir}/sentence_{i+1:02d}.mp3"
        tts.save(audio_path)
        print(f"Saved audio: {audio_path}")

In [ ]:
def create_video_from_images_and_audio(output_dir, video_path):
    """
    Create a video from images and audio files in the output directory.

    :param output_dir: The directory containing images and audio files.
    :param video_path: The path to save the output video.
    """
    image_files = sorted([f for f in os.listdir(output_dir) if f.endswith(".png") and len(f.split('_')[-1].split('.')[0]) == 2])
    audio_files = sorted([f for f in os.listdir(output_dir) if f.endswith(".mp3") and len(f.split('_')[-1].split('.')[0]) == 2])

    clips = []
    for image_file, audio_file in zip(image_files, audio_files):
        image_path = os.path.join(output_dir, image_file)
        audio_path = os.path.join(output_dir, audio_file)

        image_clip = ImageClip(image_path).set_duration(AudioFileClip(audio_path).duration)
        image_clip = image_clip.set_audio(AudioFileClip(audio_path))

        clips.append(image_clip)

    video = concatenate_videoclips(clips)
    video.write_videofile(video_path, fps=24)

In [ ]:
# Example usage
story = ("Ah, the tale of Queen Loona, a legendary ruler of a mystical kingdom hidden in the heart of the enchanted forest. Everybody in the kingdom loved her, and her wisdom and kindness were the stuff of fairy tales. Queen Loona was a woman of great beauty, with hair as golden as the sun and eyes as blue as the ocean. She was a skilled diplomat and a fierce protector of her people, and her reign was marked by peace and prosperity. One day, a young adventurer named Eira stumbled upon the hidden kingdom while on a quest to find the source of the forest's magical glow. Eira was amazed by the beauty of the kingdom and was immediately drawn to the kind and gentle queen. Queen Loona welcomed Eira with open arms and invited her to stay in the kingdom. Eira was enchanted by the queen's grace and wisdom, and she soon became a trusted advisor to the queen. Together, Queen Loona and Eira worked to improve the lives of the kingdom's people. They built schools, hospitals, and roads, and they protected the kingdom from any danger that threatened its peace. As the years passed, Queen Loona's reputation as a wise and just ruler spread throughout the land. She became known as the greatest queen the kingdom had ever known, and her legacy lived on long after she passed away. And so, the legend of Queen Loona lived on, a reminder of the power of kindness, wisdom, and compassion. Her kingdom became a beacon of hope for all those who sought a better life, and her name became synonymous with courage, grace, and beauty.")

output_dir = "story_images"  # directory to save the images and audio
video_path = "story_video.mp4"  # path to save the final video

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Generate images and audio from the story
generate_images_from_story(story, output_dir)

# Create video from images and audio
create_video_from_images_and_audio(output_dir, video_path)

# Display the generated video
from IPython.display import Video
Video(video_path)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  text_width, text_height = draw.textsize(lines, font=font)



Saved image: story_images/sentence_01.png
Saved audio: story_images/sentence_01.mp3


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  text_width, text_height = draw.textsize(lines, font=font)



Saved image: story_images/sentence_14.png
Saved audio: story_images/sentence_14.mp3
Moviepy - Building video story_video.mp4.
MoviePy - Writing audio in story_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video story_video.mp4



Moviepy - Done !
Moviepy - video ready story_video.mp4
